In [ ]:
import pandas as pd

In [ ]:
labeled_df = pd.read_csv('EIF_labeled.csv')
labeled_df.shape

In [ ]:
labeled_df = labeled_df.drop(columns='Unnamed: 0')
labeled_df.head()

In [1]:
import h2o
from h2o.estimators import H2OExtendedIsolationForestEstimator
h2o.init()
h2o_df = h2o.import_file("EIF_labeled.csv")
predictors=['country','amount','userAgent_isPc','userAgent_browserFamily','userAgent_osFamily','IBAN_countryCode','IBAN_bankCode','SIA','CAP','SAE','RAE','companyType']
eif = H2OExtendedIsolationForestEstimator()
eif.train(x = predictors,training_frame = h2o_df)
eif_result = eif.predict(h2o_df)
anomaly_score = eif_result["anomaly_score"]
mean_length = eif_result["mean_length"]

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.341-b10, mixed mode)
  Starting server from C:\Users\Robivad\AppData\Local\Programs\Python\Python310\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Robivad\AppData\Local\Temp\tmpy08ezdn_
  JVM stdout: C:\Users\Robivad\AppData\Local\Temp\tmpy08ezdn_\h2o_Robivad_started_from_python.out
  JVM stderr: C:\Users\Robivad\AppData\Local\Temp\tmpy08ezdn_\h2o_Robivad_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,1 month and 11 days
H2O_cluster_name:,H2O_from_python_Robivad_c483f2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.646 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
extendedisolationforest Model Build progress: |██████████████████████████████████| (done) 100%
extendedisolationforest prediction progress: |███████████████████████████████████| (done) 100%


In [14]:
type(anomaly_score)

h2o.frame.H2OFrame

In [ ]:
encoder_dict=pickle.load(open('enc_dict.pkl','rb'))
columns_to_label = ['country', 'userAgent_browserFamily', 'userAgent_osFamily', 'IBAN_countryCode', 'IBAN_bankCode','CAP', 'SIA', 'SAE', 'RAE','companyType']

In [ ]:
def inverse_and_scores(if_df,labeled_df,enc_dict,columns,if_model):
    df_copy=labeled_df.copy()
    scores=if_model.decision_function(if_df)
    anomaly_score=if_model.predict(if_df)

    for clmn in columns:
        df_copy[clmn]=enc_dict[clmn].inverse_transform(df_copy[clmn])

    df_copy['scores'] = scores
    df_copy['anomaly_score'] = anomaly_score

    return df_copy

df_scores=inverse_and_scores(X,labeled_df,encoder_dict,columns_to_label,model)

df_scores.head()